In [1]:
from deepface import DeepFace #facial recog.
import tensorflow as tf #for liveness check

In [2]:
import cv2, numpy ,datetime, pygame, keyboard 

import xlwings as xw #for creation of excel sheet for every day

pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model = 'liveness.model'
model = tf.keras.models.load_model(model)

In [4]:
#xlwings
#initializing workbook and worksheet
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [5]:

#xlwings

worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

In [6]:

s= 2



i = 0

entered = False
al_entered = False

In [7]:
cap = cv2.VideoCapture(0)

#pygame

pygame.init()

pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background,0,0.8)
screen.blit(background,(0,0))


#day today
t0 = datetime.date.today()
t0 = t0.day

running = True

while running:
    
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year
    
    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"
        
    screen.blit(background,(0,0))
    
    #check 
    
    if day!=t0:
        t0 = day
        worksheet = workbook.sheets.add(date)

        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i = 0
        entered = False
        al_entered = False
    
    
    
    
    
    state, frame = cap.read()
    
    if state!=True:
        break
        
    res  = DeepFace.find(frame, db_path='./Database/', enforce_detection=False, model_name='VGG-Face')
    
    if len(res[0]['identity'])>0:
        
        name = res[0]['identity'][0].split('\\')[-2].split('/')[-1]


        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        
        xmax = int(xmin + w)
        ymax = int(ymin + h)
        
        #liveness
        
        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32,32))
        img = img.astype('float')/255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = numpy.expand_dims(img, axis=0)
        
        liveness = model.predict(img)
        
        liveness = liveness[0].argmax()
        
        
        

        
        if liveness==1:
            cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
            cv2.rectangle(frame, (xmin, ymin-25),(xmax, ymin),(255,255,255),-1)
            cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)
            
            n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
            n_txtrect = n_txt.get_rect()
            n_txtrect.center =(960-200, 540//2)
            
            d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
            d_txtrect = d_txt.get_rect()
            d_txtrect.center= (960-200, (540//2)+40)
            
            t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
            t_txtrect = d_txt.get_rect()
            t_txtrect.center = (960-200, (540//2)+80)         
            
            
            screen.blit(n_txt, n_txtrect)
            screen.blit(d_txt, d_txtrect)
            screen.blit(t_txt, t_txtrect)
            
            if keyboard.is_pressed('enter'):
                
                if name not in students:
                    
                    worksheet.range(f'A{s}').value = name
                    worksheet.range(f'B{s}').value = date
                    worksheet.range(f'c{s}').value = time
                    students.append(name)
                    s+=1
                    entered =True
                else:
                    al_entered = True
                    
            if i<30 and entered==True:
                e_txt = pygame_font.render('entered', True, pg_txt, pg_bg)
                e_txtrect = e_txt.get_rect()
                e_txtrect.center = (960-200, (540//2)+120)
                screen.blit(e_txt, e_txtrect)
                i+=1

            elif i<30 and al_entered==True:
                ae_txt = pygame_font.render('already entered', True, pg_txt, pg_bg)
                ae_txtrect = ae_txt.get_rect()
                ae_txtrect.center = (960-200, (540//2)+120)
                screen.blit(ae_txt, e_txtrect)    
                i+=1

            else:
                entered=False
                al_entered = False
                i = 0

    
    
    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
    image = pygame.transform.rotozoom(image, 0 , 0.8)
    #round corners--
    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img, (255,255,255), (0,0,*size), border_radius=10)
    
    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    #round corner ---
    
    screen.blit(image, (20,80))
    
    
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    #cv2.imshow('attendance', frame)
    
    #c = cv2.waitKey(1)
    
    #if c == ord('q'):
        #break
        
cap.release()
pygame.quit()
#cv2.destroyAllWindows()

        
        


24-05-31 13:59:13 - Searching [[[155 205 223]
  [156 205 223]
  [157 206 224]
  ...
  [ 54  97 127]
  [ 53  95 124]
  [ 52  94 123]]

 [[154 204 222]
  [155 204 222]
  [155 204 222]
  ...
  [ 51  94 124]
  [ 50  92 121]
  [ 49  91 120]]

 [[163 205 224]
  [164 205 224]
  [163 204 223]
  ...
  [ 58  90 122]
  [ 57  88 119]
  [ 56  87 118]]

 ...

 [[ 71  96 112]
  [ 72  97 113]
  [ 73  98 114]
  ...
  [ 39  73  85]
  [ 38  72  85]
  [ 37  71  84]]

 [[ 70  96 112]
  [ 70  96 112]
  [ 71  97 113]
  ...
  [ 38  71  87]
  [ 37  70  85]
  [ 36  70  84]]

 [[ 70  96 112]
  [ 70  96 112]
  [ 71  97 113]
  ...
  [ 38  71  87]
  [ 36  69  84]
  [ 36  70  84]]] in 13 length datastore
24-05-31 13:59:18 - find function duration 5.089313268661499 seconds
1/1 [==============================] - 12s 12s/step
24-05-31 13:59:31 - Searching [[[135 140 139]
  [140 144 143]
  [142 144 144]
  ...
  [ 45  58  56]
  [ 44  57  55]
  [ 42  55  53]]

 [[139 144 143]
  [138 142 141]
  [138 140 140]
  ...
  [ 46  